In [2]:
from functions import mle , gradient, init_graph, matching_func, regularized_vector, sigmoid, noise_generation

import random, csv, time, os, pickle, re, math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import optimize
from scipy.stats import bernoulli
from scipy.io import savemat
from __future__ import division
from datetime import datetime

In [ ]:
def append_pair(comp, name1, name2, name_to_num):
    if comp == 'LEFT':
        return((name_to_num[name1], name_to_num[name2]))
    if comp == 'RIGHT':
        return((name_to_num[name2], name_to_num[name1]))


# Used to store video name and their corresponding index
name_to_num = dict()
# Here pairs will be a list of 5 lists correpesonding to each of the 5 traits
pairs = [[],[],[],[],[]]

j = 0
path = './AMT_results/'   # place here all csv files from AMT results
files = [f for f in os.listdir(path) if f.endswith('.csv')]
for file in files:
    print file
    with open(path+file, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader, None)
        for row in reader:      
            if not row[1] in name_to_num:
                name_to_num[row[1]] = j
                j += 1
            if not row[2] in name_to_num:
                name_to_num[row[2]] = j
                j += 1
            for i in range(3,8):
                pair = append_pair(row[i],row[1],row[2],name_to_num)
                if pair != None:
                    pairs[i-3].append(pair)
video_num = j


video_num
len(pairs[0])

In [22]:
pickle.dump( dict({'pairs':pairs}), open( 'AMT_pairs.p', "wb" ) )

In [4]:
w = np.ones(10000)

start_time = time.time()

res = optimize.minimize(mle, w, 
                        method='Newton-CG',
                        jac=gradient,
                        args=(pairs[0],1,),
                        tol = 10,
                        options={'disp': False})

pickle.dump( dict({'x':res.x}), open( 'AMT_0.p', "wb" ) )
print time.time() - start_time

3088.27992511


In [5]:
w = np.ones(10000)

start_time = time.time()

res = optimize.minimize(mle, w, 
                        method='Newton-CG',
                        jac=gradient,
                        args=(pairs[1],1,),
                        tol = 10,
                        options={'disp': False})

pickle.dump( dict({'x':res.x}), open( 'AMT_1.p', "wb" ) )
print time.time() - start_time

3050.96932912


In [6]:
w = np.ones(10000)

start_time = time.time()

res = optimize.minimize(mle, w, 
                        method='Newton-CG',
                        jac=gradient,
                        args=(pairs[2],1,),
                        tol = 10,
                        options={'disp': False})

pickle.dump( dict({'x':res.x}), open( 'AMT_2.p', "wb" ) )
print time.time() - start_time

3248.16994596


In [7]:
w = np.ones(10000)

start_time = time.time()

res = optimize.minimize(mle, w, 
                        method='Newton-CG',
                        jac=gradient,
                        args=(pairs[3],1,),
                        tol = 10,
                        options={'disp': False})

pickle.dump( dict({'x':res.x}), open( 'AMT_3.p', "wb" ) )
print time.time() - start_time

3535.49453211


In [8]:
w = np.ones(10000)

start_time = time.time()

res = optimize.minimize(mle, w, 
                        method='Newton-CG',
                        jac=gradient,
                        args=(pairs[4],1,),
                        tol = 10,
                        options={'disp': False})

pickle.dump( dict({'x':res.x}), open( 'AMT_4.p', "wb" ) )
print time.time() - start_time

3913.07666302


# Below is reconstruction accuracy calculation

In [57]:
scores = pickle.load( open( 'AMT_4.p', "rb" ) )
pairs = pickle.load( open( 'AMT_pairs.p', "rb" ) )
pairs = pairs['pairs'][4]
scoresx = scores['x']
np.std(scores['x'])/2

0.20630241285824419

In [58]:
discount =0
cool = 0
threshold = 0.206
weight = 0.5
for i in range(len(pairs)):
    if (scoresx[pairs[i][0]]-scoresx[pairs[i][1]] >= 0) and (scoresx[pairs[i][0]]-scoresx[pairs[i][1]] < threshold):
        cool += weight
        discount+=1
    if (scoresx[pairs[i][0]]-scoresx[pairs[i][1]] < 0) and (scoresx[pairs[i][1]]-scoresx[pairs[i][0]] < threshold):
        cool += weight
        discount+=1
    if (scoresx[pairs[i][0]]-scoresx[pairs[i][1]] >= threshold):
        cool += 1
    
print cool/(321684-discount*(1-weight))

0.674939053077


In [59]:
scoresx = scores['x']
discount =0
cool = 0

for i in range(len(pairs)):
    if (scoresx[pairs[i][0]]-scoresx[pairs[i][1]] > 0):
        cool += 1

print cool/(321684)

0.585587097897


# Below is naive counting method as a baseline calculation

In [4]:
scores = pickle.load( open( 'AMT_0.p', "rb" ) )
pairs_all = pickle.load( open( 'AMT_pairs.p', "rb" ) )


In [7]:
scores = [np.zeros(10000),np.zeros(10000),np.zeros(10000),np.zeros(10000),np.zeros(10000)]
for trait in range(5):
    pairs = pairs_all['pairs'][trait]
    for pair in pairs:
        scores[trait][pair[0]] += 1
    correct = 0
    for pair in pairs:
        if scores[trait][pair[0]] > scores[trait][pair[1]]:
            correct += 1
    print 'On trait',trait,'and accuracy is:', correct/len(pairs)

On trait 0 and accuracy is: 0.627127864612
On trait 1 and accuracy is: 0.58064192326
On trait 2 and accuracy is: 0.629143041093
On trait 3 and accuracy is: 0.608602894001
On trait 4 and accuracy is: 0.598095039884
